# Neural Network Modeling

In [4]:
import time
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras.models
from tensorflow.keras import models, layers
from tensorflow.keras import callbacks

from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_curve, roc_auc_score, log_loss, accuracy_score, classification_report


In [9]:
# Import Response Variable
train_swn_target = pd.read_csv('./data/train_swn_target.csv', header = None)
train_afn_target = pd.read_csv('./data/train_afn_target.csv', header = None)
val_swn_target = pd.read_csv('./data/val_swn_target.csv', header = None)
val_afn_target = pd.read_csv('./data/val_afn_target.csv', header = None)

# Import Training Data
dm_train_vecs = pd.read_csv('./data/dm_train_vecs.csv', header = None)
dm_mean_train_vecs = pd.read_csv('./data/dm_mean_train_vecs.csv', header = None)
dm_concat_train_vecs = pd.read_csv('./data/dm_concat_train_vecs.csv', header = None)
dbow_train_vecs = pd.read_csv('./data/dbow_train_vecs.csv', header = None)

tf_uni_train_vecs = pd.read_csv('./data/tf_uni_train_vecs.csv', header = None)
tf_bi_train_vecs = pd.read_csv('./data/tf_bi_train_vecs.csv', header = None)
tf_tri_train_vecs = pd.read_csv('./data/tf_tri_train_vecs.csv', header = None)

# Import Validation Data
dm_val_vecs = pd.read_csv('./data/dm_val_vecs.csv', header = None)
dm_mean_val_vecs = pd.read_csv('./data/dm_mean_val_vecs.csv', header = None)
dm_concat_val_vecs = pd.read_csv('./data/dm_concat_val_vecs.csv', header = None)
dbow_val_vecs = pd.read_csv('./data/dbow_val_vecs.csv', header = None)

tf_uni_val_vecs = pd.read_csv('./data/tf_uni_val_vecs.csv', header = None)
tf_bi_val_vecs = pd.read_csv('./data/tf_bi_val_vecs.csv', header = None)
tf_tri_val_vecs = pd.read_csv('./data/tf_tri_val_vecs.csv', header = None)


In [10]:
nn_train_swn_target = train_swn_target + 1
nn_val_swn_target = val_swn_target + 1

nn_train_afn_target = train_afn_target + 1
nn_val_afn_target = val_afn_target + 1

In [11]:
import os
root_logdir = os.path.join(os.curdir, "my_logs")


def get_run_logdir(model_name):
    run_id = time.strftime('run_%Y_%m_%d-%H_%M_%S_') + str(model_name)
    return os.path.join(root_logdir, run_id)

In [12]:
def classif_reports(y_true, y_pred):
    print('Accuracy Score:', accuracy_score(y_true, y_pred))
    # print('\nNum of Values per Class:\n', pd.Series(swn_preds).value_counts())
    swn_report = classification_report(y_true, y_pred, labels = [0, 1, -1], target_names = ['Neut', 'Pos', 'Neg'])
    print('\nClassification Report',swn_report)
    conf_matrix = pd.DataFrame(confusion_matrix(y_true, y_pred, normalize = 'true'), index = ['True Neg', 'True Neut', 'True Pos'], columns = ['Pred Neg', 'Pred Neut', 'Pred Pos'])
    return conf_matrix

## Fully Connected Neural Network

In [ ]:
def neural_network(X, y, val_data = None, num_layers = 2, neurons = 100, dropout_rate = 0.5, 
                    model_name = 'neural_network', optimizer = 'adam', patience = 10):

    model = models.Sequential()
    model.add(layers.InputLayer(input_shape = X.shape[1:]))
    for layer in range(num_layers):
        model.add(layers.Dropout(rate = dropout_rate))
        model.add(layers.Dense(neurons, activation = 'relu'))
    model.add(layers.Dense(3, activation = 'softmax'))

    model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

    run_log_dir = get_run_logdir(model_name = model_name + '_layrs_' + str(num_layers) + '_neur_' + str(neurons) + '_drop_' + str(dropout_rate))

    checkpoint_cb = callbacks.ModelCheckpoint('./models/nn models/' + model_name +'.h5', save_best_only = True)
    early_stop_cb = callbacks.EarlyStopping(patience = patience, restore_best_weights = True)
    tensorboard_cb = callbacks.TensorBoard(run_log_dir)

    start_time = time.time()
    history = model.fit(X, y, validation_data = val_data, 
                        epochs = 100, batch_size = 32, 
                        verbose = 2, callbacks = [checkpoint_cb, early_stop_cb, tensorboard_cb])
    print('---- Total time to execute: %s seconds ----' %(time.time() - start_time)) 

    return model, history

### DM Model

#### SentiWordNet

In [ ]:
dm_NN_swn, history = neural_network(dm_train_vecs, nn_train_swn_target, val_data = (dm_val_vecs, nn_val_swn_target), 
                                        num_layers = 4, dropout_rate = 0, model_name = 'DM_NN_swn')

Epoch 1/100
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1137/1137 - 3s - loss: 1.0067 - accuracy: 0.4892 - val_loss: 1.0863 - val_accuracy: 0.4652
Epoch 2/100
1137/1137 - 3s - loss: 0.9687 - accuracy: 0.5218 - val_loss: 1.0865 - val_accuracy: 0.4648
Epoch 3/100
1137/1137 - 3s - loss: 0.9414 - accuracy: 0.5426 - val_loss: 1.0436 - val_accuracy: 0.4894
Epoch 4/100
1137/1137 - 3s - loss: 0.9156 - accuracy: 0.5654 - val_loss: 1.0726 - val_accuracy: 0.4950
Epoch 5/100
1137/1137 - 3s - loss: 0.8855 - accuracy: 0.5854 - val_loss: 1.1266 - val_accuracy: 0.4904
Epoch 6/100
1137/1137 - 3s - loss: 0.8558 - accuracy: 0.6056 - val_loss: 1.1007 - val_accuracy: 0.5010
Epoch 7/100
1137/1137 - 3s - loss: 0.8236 - accuracy: 0.6274 - val_loss: 1.1905 - val_accuracy: 0.4884
Epoch 8/100
1137/1137 - 3s - loss: 0.7916 - accuracy: 0.6462 - val_loss: 1.1996 - val_accuracy: 0.4980
Epoch 9/100
1137/1137 - 3s - loss: 0.7589 - accuracy: 0.6665 - val_loss: 1.1545 - val_accuracy: 0.5040
E

In [ ]:
dm_NN_swn_train_preds = np.argmax(dm_NN_swn.predict(dm_train_vecs), axis = -1) - 1
classif_reports(train_swn_target, dm_NN_swn_train_preds)

Accuracy Score: 0.5708235941152207

Classification Report               precision    recall  f1-score   support

        Neut       0.52      0.56      0.54      9632
         Pos       0.61      0.58      0.59     14691
         Neg       0.58      0.56      0.57     12042

    accuracy                           0.57     36365
   macro avg       0.57      0.57      0.57     36365
weighted avg       0.57      0.57      0.57     36365



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.564607,0.163843,0.271550
True Neut,0.198297,0.562292,0.239410
True Pos,0.206725,0.211762,0.581512


In [ ]:
dm_NN_swn_val_preds = np.argmax(dm_NN_swn.predict(dm_val_vecs), axis = -1) - 1
classif_reports(val_swn_target, dm_NN_swn_val_preds)

Accuracy Score: 0.4894

Classification Report               precision    recall  f1-score   support

        Neut       0.52      0.18      0.26      1313
         Pos       0.49      0.65      0.56      2005
         Neg       0.48      0.55      0.51      1682

    accuracy                           0.49      5000
   macro avg       0.50      0.46      0.44      5000
weighted avg       0.49      0.49      0.47      5000



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.545184,0.044590,0.410226
True Neut,0.329779,0.177456,0.492765
True Pos,0.282294,0.070823,0.646883


In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir=./my_logs --port=6006

#### Afinn

In [ ]:
dm_nn_afn, history = neural_network(dm_train_vecs, nn_train_afn_target, val_data = (dm_val_vecs, nn_val_afn_target),
                             model_name = 'DM_AFN_NN', num_layers = 4, neurons = 150, dropout_rate = 0.1)

Epoch 1/100
1137/1137 - 3s - loss: 1.0127 - accuracy: 0.4935 - val_loss: 1.0501 - val_accuracy: 0.4776
Epoch 2/100
1137/1137 - 3s - loss: 0.9758 - accuracy: 0.5311 - val_loss: 1.0008 - val_accuracy: 0.5116
Epoch 3/100
1137/1137 - 3s - loss: 0.9558 - accuracy: 0.5426 - val_loss: 0.9840 - val_accuracy: 0.5178
Epoch 4/100
1137/1137 - 3s - loss: 0.9434 - accuracy: 0.5527 - val_loss: 0.9922 - val_accuracy: 0.5140
Epoch 5/100
1137/1137 - 3s - loss: 0.9298 - accuracy: 0.5629 - val_loss: 0.9784 - val_accuracy: 0.5202
Epoch 6/100
1137/1137 - 3s - loss: 0.9184 - accuracy: 0.5690 - val_loss: 0.9669 - val_accuracy: 0.5300
Epoch 7/100
1137/1137 - 3s - loss: 0.9069 - accuracy: 0.5780 - val_loss: 0.9726 - val_accuracy: 0.5232
Epoch 8/100
1137/1137 - 3s - loss: 0.8969 - accuracy: 0.5839 - val_loss: 1.0056 - val_accuracy: 0.5166
Epoch 9/100
1137/1137 - 3s - loss: 0.8904 - accuracy: 0.5885 - val_loss: 0.9779 - val_accuracy: 0.5318
Epoch 10/100
1137/1137 - 3s - loss: 0.8836 - accuracy: 0.5918 - val_loss:

In [ ]:
NN_afn_train_preds = np.argmax(dm_nn_afn.predict(dm_train_vecs), axis = -1) - 1
classif_reports(train_afn_target, NN_afn_train_preds)

Accuracy Score: 0.6235116183143132

Classification Report               precision    recall  f1-score   support

        Neut       0.56      0.67      0.61     12086
         Pos       0.65      0.60      0.62     11581
         Neg       0.67      0.61      0.64     12698

    accuracy                           0.62     36365
   macro avg       0.63      0.62      0.62     36365
weighted avg       0.63      0.62      0.62     36365



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.608049,0.250512,0.141440
True Neut,0.179629,0.665315,0.155055
True Pos,0.137467,0.265694,0.596840


In [ ]:
NN_afn_val_preds = np.argmax(dm_nn_afn.predict(dm_val_vecs), axis = -1) - 1
classif_reports(val_afn_target, NN_afn_val_preds)

Accuracy Score: 0.53

Classification Report               precision    recall  f1-score   support

        Neut       0.57      0.34      0.43      1712
         Pos       0.49      0.64      0.55      1529
         Neg       0.55      0.62      0.58      1759

    accuracy                           0.53      5000
   macro avg       0.54      0.53      0.52      5000
weighted avg       0.54      0.53      0.52      5000



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.619670,0.122229,0.258101
True Neut,0.322430,0.339953,0.337617
True Pos,0.215173,0.145193,0.639634


### DM Mean Model

#### SentiWordNet

In [ ]:
dm_mean_nn_swn, history = neural_network(dm_mean_train_vecs, nn_train_swn_target, val_data = (dm_mean_val_vecs, nn_val_swn_target), model_name = 'dm_mean_swn_nn',
                                        neurons = 130, num_layers = 3, dropout_rate = 0.1)

Epoch 1/100
1137/1137 - 3s - loss: 1.0196 - accuracy: 0.4766 - val_loss: 1.0376 - val_accuracy: 0.4764
Epoch 2/100
1137/1137 - 3s - loss: 0.9911 - accuracy: 0.5055 - val_loss: 1.0548 - val_accuracy: 0.4690
Epoch 3/100
1137/1137 - 3s - loss: 0.9762 - accuracy: 0.5179 - val_loss: 1.0340 - val_accuracy: 0.4882
Epoch 4/100
1137/1137 - 3s - loss: 0.9666 - accuracy: 0.5226 - val_loss: 1.0452 - val_accuracy: 0.4896
Epoch 5/100
1137/1137 - 3s - loss: 0.9573 - accuracy: 0.5298 - val_loss: 1.0230 - val_accuracy: 0.5024
Epoch 6/100
1137/1137 - 3s - loss: 0.9468 - accuracy: 0.5361 - val_loss: 1.0197 - val_accuracy: 0.4960
Epoch 7/100
1137/1137 - 3s - loss: 0.9380 - accuracy: 0.5475 - val_loss: 1.0556 - val_accuracy: 0.4942
Epoch 8/100
1137/1137 - 3s - loss: 0.9310 - accuracy: 0.5482 - val_loss: 1.0109 - val_accuracy: 0.5072
Epoch 9/100
1137/1137 - 3s - loss: 0.9245 - accuracy: 0.5538 - val_loss: 1.0299 - val_accuracy: 0.5054
Epoch 10/100
1137/1137 - 3s - loss: 0.9187 - accuracy: 0.5587 - val_loss:

In [ ]:
dm_mean_train_preds_swn = np.argmax(dm_mean_nn_swn.predict(dm_mean_train_vecs), axis = -1) - 1
classif_reports(train_swn_target, dm_mean_train_preds_swn)

Accuracy Score: 0.5948851918053073

Classification Report               precision    recall  f1-score   support

        Neut       0.53      0.60      0.56      9632
         Pos       0.60      0.68      0.64     14691
         Neg       0.66      0.49      0.56     12042

    accuracy                           0.59     36365
   macro avg       0.60      0.59      0.59     36365
weighted avg       0.60      0.59      0.59     36365



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.491779,0.192576,0.315645
True Neut,0.116487,0.599772,0.283742
True Pos,0.133347,0.190457,0.676196


In [ ]:
dm_mean_val_preds_swn = np.argmax(dm_mean_nn_swn.predict(dm_mean_val_vecs), axis = -1) - 1
classif_reports(val_swn_target, dm_mean_val_preds_swn)

Accuracy Score: 0.5072

Classification Report               precision    recall  f1-score   support

        Neut       0.52      0.26      0.35      1313
         Pos       0.50      0.71      0.59      2005
         Neg       0.51      0.46      0.49      1682

    accuracy                           0.51      5000
   macro avg       0.51      0.48      0.47      5000
weighted avg       0.51      0.51      0.49      5000



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.460761,0.085612,0.453627
True Neut,0.239909,0.258949,0.501142
True Pos,0.207481,0.083791,0.708728


#### Afin

In [ ]:
dm_mean_nn_afn, history = neural_network(dm_mean_train_vecs, nn_train_afn_target, val_data = (dm_mean_val_vecs, nn_val_afn_target), model_name = 'dm_mean_afn_nn',
                                        neurons = 100, num_layers = 4, dropout_rate = 0)

Epoch 1/100
1137/1137 - 4s - loss: 0.9905 - accuracy: 0.5087 - val_loss: 1.0140 - val_accuracy: 0.5000
Epoch 2/100
1137/1137 - 3s - loss: 0.9367 - accuracy: 0.5564 - val_loss: 1.0676 - val_accuracy: 0.4834
Epoch 3/100
1137/1137 - 3s - loss: 0.9034 - accuracy: 0.5797 - val_loss: 1.0491 - val_accuracy: 0.4964
Epoch 4/100
1137/1137 - 3s - loss: 0.8730 - accuracy: 0.6013 - val_loss: 1.0382 - val_accuracy: 0.5184
Epoch 5/100
1137/1137 - 3s - loss: 0.8414 - accuracy: 0.6174 - val_loss: 1.0261 - val_accuracy: 0.5238
Epoch 6/100
1137/1137 - 3s - loss: 0.8114 - accuracy: 0.6357 - val_loss: 1.0859 - val_accuracy: 0.5204
Epoch 7/100
1137/1137 - 3s - loss: 0.7814 - accuracy: 0.6554 - val_loss: 1.0526 - val_accuracy: 0.5228
Epoch 8/100
1137/1137 - 3s - loss: 0.7489 - accuracy: 0.6755 - val_loss: 1.1082 - val_accuracy: 0.5190
Epoch 9/100
1137/1137 - 3s - loss: 0.7174 - accuracy: 0.6921 - val_loss: 1.2290 - val_accuracy: 0.5028
Epoch 10/100
1137/1137 - 3s - loss: 0.6868 - accuracy: 0.7083 - val_loss:

In [ ]:
dm_mean_train_preds_afn = np.argmax(dm_mean_nn_afn.predict(dm_mean_train_vecs), axis = -1) - 1
classif_reports(train_afn_target, dm_mean_train_preds_afn)

Accuracy Score: 0.5528942664650076

Classification Report               precision    recall  f1-score   support

        Neut       0.49      0.65      0.56     12086
         Pos       0.62      0.44      0.51     11581
         Neg       0.59      0.57      0.58     12698

    accuracy                           0.55     36365
   macro avg       0.57      0.55      0.55     36365
weighted avg       0.57      0.55      0.55     36365



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.569854,0.309498,0.120649
True Neut,0.223895,0.646533,0.129571
True Pos,0.201537,0.361886,0.436577


In [ ]:
dm_mean_val_preds_afn = np.argmax(dm_mean_nn_afn.predict(dm_mean_val_vecs), axis = -1) - 1
classif_reports(val_afn_target, dm_mean_val_preds_afn)

Accuracy Score: 0.5

Classification Report               precision    recall  f1-score   support

        Neut       0.51      0.35      0.41      1712
         Pos       0.48      0.54      0.51      1529
         Neg       0.51      0.61      0.56      1759

    accuracy                           0.50      5000
   macro avg       0.50      0.50      0.49      5000
weighted avg       0.50      0.50      0.49      5000



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.614554,0.163161,0.222285
True Neut,0.360397,0.345794,0.293808
True Pos,0.277305,0.181818,0.540876


### DM Concat

#### Sentiword

In [ ]:
dmc_nn_swn, history = neural_network(dm_concat_train_vecs, nn_train_swn_target, val_data = (dm_concat_val_vecs, nn_val_swn_target), 
                                        model_name = 'dmc_nn_swn', num_layers = 3, neurons = 200, dropout_rate = 0.15)

Epoch 1/100
1137/1137 - 4s - loss: 1.0273 - accuracy: 0.4680 - val_loss: 1.0544 - val_accuracy: 0.4580
Epoch 2/100
1137/1137 - 3s - loss: 1.0016 - accuracy: 0.4981 - val_loss: 1.0423 - val_accuracy: 0.4726
Epoch 3/100
1137/1137 - 3s - loss: 0.9891 - accuracy: 0.5067 - val_loss: 1.0498 - val_accuracy: 0.4718
Epoch 4/100
1137/1137 - 4s - loss: 0.9798 - accuracy: 0.5145 - val_loss: 1.0097 - val_accuracy: 0.4814
Epoch 5/100
1137/1137 - 4s - loss: 0.9732 - accuracy: 0.5194 - val_loss: 1.0325 - val_accuracy: 0.4870
Epoch 6/100
1137/1137 - 4s - loss: 0.9651 - accuracy: 0.5215 - val_loss: 1.0258 - val_accuracy: 0.4888
Epoch 7/100
1137/1137 - 3s - loss: 0.9594 - accuracy: 0.5304 - val_loss: 1.0152 - val_accuracy: 0.5006
Epoch 8/100
1137/1137 - 3s - loss: 0.9512 - accuracy: 0.5369 - val_loss: 1.0037 - val_accuracy: 0.4986
Epoch 9/100
1137/1137 - 3s - loss: 0.9510 - accuracy: 0.5354 - val_loss: 1.0028 - val_accuracy: 0.4988
Epoch 10/100
1137/1137 - 3s - loss: 0.9430 - accuracy: 0.5396 - val_loss:

In [ ]:
dmc_nn_swn_train_preds = np.argmax(dmc_nn_swn.predict(dm_concat_train_vecs), axis = -1) - 1
classif_reports(train_swn_target, dmc_nn_swn_train_preds)

Accuracy Score: 0.5918053073009762

Classification Report               precision    recall  f1-score   support

        Neut       0.52      0.62      0.56      9632
         Pos       0.63      0.61      0.62     14691
         Neg       0.62      0.55      0.58     12042

    accuracy                           0.59     36365
   macro avg       0.59      0.59      0.59     36365
weighted avg       0.60      0.59      0.59     36365



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.554642,0.196230,0.249128
True Neut,0.160818,0.615552,0.223630
True Pos,0.176366,0.216936,0.606698


In [ ]:
dmc_nn_swn_val_preds = np.argmax(dmc_nn_swn.predict(dm_concat_val_vecs), axis = -1) - 1
classif_reports(val_swn_target, dmc_nn_swn_val_preds)

Accuracy Score: 0.4988

Classification Report               precision    recall  f1-score   support

        Neut       0.51      0.24      0.32      1313
         Pos       0.50      0.67      0.58      2005
         Neg       0.49      0.50      0.49      1682

    accuracy                           0.50      5000
   macro avg       0.50      0.47      0.46      5000
weighted avg       0.50      0.50      0.48      5000



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.498811,0.076694,0.424495
True Neut,0.300076,0.238385,0.461538
True Pos,0.243392,0.087282,0.669327


#### Afin

In [ ]:
dmc_nn_afn, history = neural_network(dm_concat_train_vecs, nn_train_afn_target, val_data = (dm_concat_val_vecs, nn_val_afn_target), 
                                        model_name = 'dmc_nn_afn', num_layers = 3, neurons = 200, dropout_rate = 0.15)

Epoch 1/100
1137/1137 - 4s - loss: 1.0193 - accuracy: 0.4894 - val_loss: 1.0216 - val_accuracy: 0.5010
Epoch 2/100
1137/1137 - 4s - loss: 0.9850 - accuracy: 0.5223 - val_loss: 0.9994 - val_accuracy: 0.5004
Epoch 3/100
1137/1137 - 4s - loss: 0.9692 - accuracy: 0.5335 - val_loss: 0.9974 - val_accuracy: 0.5022
Epoch 4/100
1137/1137 - 3s - loss: 0.9544 - accuracy: 0.5444 - val_loss: 0.9753 - val_accuracy: 0.5220
Epoch 5/100
1137/1137 - 4s - loss: 0.9443 - accuracy: 0.5506 - val_loss: 0.9608 - val_accuracy: 0.5304
Epoch 6/100
1137/1137 - 4s - loss: 0.9354 - accuracy: 0.5514 - val_loss: 0.9501 - val_accuracy: 0.5484
Epoch 7/100
1137/1137 - 3s - loss: 0.9305 - accuracy: 0.5582 - val_loss: 0.9571 - val_accuracy: 0.5402
Epoch 8/100
1137/1137 - 4s - loss: 0.9234 - accuracy: 0.5645 - val_loss: 0.9637 - val_accuracy: 0.5284
Epoch 9/100
1137/1137 - 4s - loss: 0.9160 - accuracy: 0.5699 - val_loss: 0.9583 - val_accuracy: 0.5474
Epoch 10/100
1137/1137 - 3s - loss: 0.9119 - accuracy: 0.5738 - val_loss:

In [ ]:
dmc_nn_afn_train_preds = np.argmax(dmc_nn_afn.predict(dm_concat_train_vecs), axis = -1) - 1
classif_reports(train_afn_target, dmc_nn_afn_train_preds)

Accuracy Score: 0.631623814106971

Classification Report               precision    recall  f1-score   support

        Neut       0.53      0.80      0.64     12086
         Pos       0.73      0.54      0.62     11581
         Neg       0.73      0.55      0.63     12698

    accuracy                           0.63     36365
   macro avg       0.67      0.63      0.63     36365
weighted avg       0.67      0.63      0.63     36365



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.549378,0.349976,0.100646
True Neut,0.110293,0.803492,0.086215
True Pos,0.104309,0.353251,0.542440


In [ ]:
dmc_nn_afn_val_preds = np.argmax(dmc_nn_afn.predict(dm_concat_val_vecs), axis = -1) - 1
classif_reports(val_afn_target, dmc_nn_afn_val_preds)

Accuracy Score: 0.5486

Classification Report               precision    recall  f1-score   support

        Neut       0.53      0.50      0.51      1712
         Pos       0.52      0.57      0.55      1529
         Neg       0.59      0.57      0.58      1759

    accuracy                           0.55      5000
   macro avg       0.55      0.55      0.55      5000
weighted avg       0.55      0.55      0.55      5000



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.574758,0.219443,0.205799
True Neut,0.241822,0.498832,0.259346
True Pos,0.179856,0.245912,0.574232


### DBOW Model

#### Sentiword

In [ ]:
dbow_nn_swn, history = neural_network(dbow_train_vecs, nn_train_swn_target, val_data = (dbow_val_vecs, nn_val_swn_target), 
                                model_name = 'dbow_nn_swn', dropout_rate = 0.1, neurons = 100, num_layers = 3)

Epoch 1/100
1137/1137 - 3s - loss: 1.0264 - accuracy: 0.4704 - val_loss: 1.0552 - val_accuracy: 0.4696
Epoch 2/100
1137/1137 - 3s - loss: 0.9971 - accuracy: 0.4989 - val_loss: 1.0383 - val_accuracy: 0.4796
Epoch 3/100
1137/1137 - 3s - loss: 0.9843 - accuracy: 0.5079 - val_loss: 1.0295 - val_accuracy: 0.4918
Epoch 4/100
1137/1137 - 3s - loss: 0.9759 - accuracy: 0.5145 - val_loss: 1.0463 - val_accuracy: 0.4820
Epoch 5/100
1137/1137 - 3s - loss: 0.9655 - accuracy: 0.5250 - val_loss: 1.0269 - val_accuracy: 0.4962
Epoch 6/100
1137/1137 - 3s - loss: 0.9572 - accuracy: 0.5308 - val_loss: 1.0262 - val_accuracy: 0.4974
Epoch 7/100
1137/1137 - 3s - loss: 0.9532 - accuracy: 0.5349 - val_loss: 1.0353 - val_accuracy: 0.4896
Epoch 8/100
1137/1137 - 3s - loss: 0.9472 - accuracy: 0.5357 - val_loss: 1.0365 - val_accuracy: 0.4896
Epoch 9/100
1137/1137 - 3s - loss: 0.9406 - accuracy: 0.5402 - val_loss: 1.0391 - val_accuracy: 0.4974
Epoch 10/100
1137/1137 - 3s - loss: 0.9361 - accuracy: 0.5459 - val_loss:

In [ ]:
dbow_train_preds_swn = np.argmax(dbow_nn_swn.predict(dbow_train_vecs), axis = -1) - 1
classif_reports(train_swn_target, dbow_train_preds_swn)

Accuracy Score: 0.5959576515880655

Classification Report               precision    recall  f1-score   support

        Neut       0.53      0.61      0.57      9632
         Pos       0.60      0.68      0.64     14691
         Neg       0.66      0.48      0.56     12042

    accuracy                           0.60     36365
   macro avg       0.60      0.59      0.59     36365
weighted avg       0.60      0.60      0.59     36365



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.480568,0.201379,0.318053
True Neut,0.104651,0.612126,0.283223
True Pos,0.130216,0.189844,0.679940


In [ ]:
dbow_val_preds_swn = np.argmax(dbow_nn_swn.predict(dbow_val_vecs), axis = -1) - 1
classif_reports(val_swn_target, dbow_val_preds_swn)

Accuracy Score: 0.5046

Classification Report               precision    recall  f1-score   support

        Neut       0.55      0.20      0.30      1313
         Pos       0.49      0.75      0.59      2005
         Neg       0.52      0.45      0.48      1682

    accuracy                           0.50      5000
   macro avg       0.52      0.47      0.46      5000
weighted avg       0.52      0.50      0.48      5000



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.450654,0.060048,0.489298
True Neut,0.245240,0.204874,0.549886
True Pos,0.194015,0.059850,0.746135


#### Afin

In [ ]:
dbow_nn_afn, history = neural_network(dbow_train_vecs, nn_train_afn_target, val_data = (dbow_val_vecs, nn_val_afn_target), 
                                model_name = 'dbow_nn_afn', dropout_rate = 0.1, neurons = 100, num_layers = 3)

Epoch 1/100
1137/1137 - 3s - loss: 1.0144 - accuracy: 0.4918 - val_loss: 1.0416 - val_accuracy: 0.4908
Epoch 2/100
1137/1137 - 3s - loss: 0.9789 - accuracy: 0.5231 - val_loss: 1.0164 - val_accuracy: 0.5034
Epoch 3/100
1137/1137 - 3s - loss: 0.9647 - accuracy: 0.5350 - val_loss: 0.9931 - val_accuracy: 0.4954
Epoch 4/100
1137/1137 - 3s - loss: 0.9530 - accuracy: 0.5454 - val_loss: 1.0117 - val_accuracy: 0.5162
Epoch 5/100
1137/1137 - 3s - loss: 0.9399 - accuracy: 0.5552 - val_loss: 0.9877 - val_accuracy: 0.5190
Epoch 6/100
1137/1137 - 3s - loss: 0.9301 - accuracy: 0.5608 - val_loss: 0.9796 - val_accuracy: 0.5194
Epoch 7/100
1137/1137 - 3s - loss: 0.9246 - accuracy: 0.5656 - val_loss: 0.9803 - val_accuracy: 0.5262
Epoch 8/100
1137/1137 - 3s - loss: 0.9155 - accuracy: 0.5674 - val_loss: 0.9736 - val_accuracy: 0.5340
Epoch 9/100
1137/1137 - 3s - loss: 0.9110 - accuracy: 0.5728 - val_loss: 0.9786 - val_accuracy: 0.5224
Epoch 10/100
1137/1137 - 3s - loss: 0.9039 - accuracy: 0.5741 - val_loss:

In [ ]:
dbow_train_preds_afn = np.argmax(dbow_nn_afn.predict(dbow_train_vecs), axis = -1) - 1
classif_reports(train_afn_target, dbow_train_preds_afn)

Accuracy Score: 0.6488381685686787

Classification Report               precision    recall  f1-score   support

        Neut       0.59      0.67      0.63     12086
         Pos       0.73      0.55      0.63     11581
         Neg       0.65      0.72      0.68     12698

    accuracy                           0.65     36365
   macro avg       0.66      0.65      0.65     36365
weighted avg       0.66      0.65      0.65     36365



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.717121,0.194440,0.088439
True Neut,0.228860,0.669618,0.101522
True Pos,0.176928,0.270788,0.552284


In [ ]:
dbow_val_preds_afn = np.argmax(dbow_nn_afn.predict(dbow_val_vecs), axis = -1) - 1
classif_reports(val_afn_target, dbow_val_preds_afn)

Accuracy Score: 0.546

Classification Report               precision    recall  f1-score   support

        Neut       0.61      0.35      0.45      1712
         Pos       0.53      0.56      0.55      1529
         Neg       0.53      0.72      0.61      1759

    accuracy                           0.55      5000
   macro avg       0.56      0.55      0.53      5000
weighted avg       0.56      0.55      0.53      5000



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.719727,0.097783,0.182490
True Neut,0.397196,0.352220,0.250584
True Pos,0.294310,0.142577,0.563113


## SentiWordNet

### TF - IDF Unigram

In [ ]:
tf_uni_nn_swn, history = neural_network(tf_uni_train_vecs, nn_train_swn_target, val_data = (tf_uni_val_vecs, nn_val_swn_target), 
                                model_name = 'tf_uni_nn_swn', dropout_rate = 0.1, neurons = 100, num_layers = 3, patience = 20)

Epoch 1/100
1137/1137 - 4s - loss: 0.9170 - accuracy: 0.5693 - val_loss: 0.8432 - val_accuracy: 0.6338
Epoch 2/100
1137/1137 - 3s - loss: 0.8282 - accuracy: 0.6365 - val_loss: 0.8169 - val_accuracy: 0.6420
Epoch 3/100
1137/1137 - 3s - loss: 0.7736 - accuracy: 0.6678 - val_loss: 0.7925 - val_accuracy: 0.6614
Epoch 4/100
1137/1137 - 3s - loss: 0.7218 - accuracy: 0.6944 - val_loss: 0.7777 - val_accuracy: 0.6700
Epoch 5/100
1137/1137 - 3s - loss: 0.6756 - accuracy: 0.7188 - val_loss: 0.7768 - val_accuracy: 0.6662
Epoch 6/100
1137/1137 - 3s - loss: 0.6364 - accuracy: 0.7380 - val_loss: 0.7678 - val_accuracy: 0.6800
Epoch 7/100
1137/1137 - 3s - loss: 0.6044 - accuracy: 0.7508 - val_loss: 0.7825 - val_accuracy: 0.6854
Epoch 8/100
1137/1137 - 3s - loss: 0.5796 - accuracy: 0.7624 - val_loss: 0.7710 - val_accuracy: 0.6900
Epoch 9/100
1137/1137 - 3s - loss: 0.5616 - accuracy: 0.7701 - val_loss: 0.7630 - val_accuracy: 0.6890
Epoch 10/100
1137/1137 - 3s - loss: 0.5462 - accuracy: 0.7729 - val_loss:

In [ ]:
tfuni_nn_swn_train_preds = np.argmax(tf_uni_nn_swn.predict(tf_uni_train_vecs), axis = -1) - 1
classif_reports(train_swn_target, tfuni_nn_swn_train_preds)

Accuracy Score: 0.8660525230303864

Classification Report               precision    recall  f1-score   support

        Neut       0.76      0.88      0.82      9632
         Pos       0.89      0.88      0.89     14691
         Neg       0.94      0.84      0.88     12042

    accuracy                           0.87     36365
   macro avg       0.86      0.87      0.86     36365
weighted avg       0.87      0.87      0.87     36365



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.835326,0.100482,0.064192
True Neut,0.035507,0.881541,0.082953
True Pos,0.023348,0.095569,0.881084


In [ ]:
tfuni_nn_swn_val_preds = np.argmax(tf_uni_nn_swn.predict(tf_uni_val_vecs), axis = -1) - 1
classif_reports(val_swn_target, tfuni_nn_swn_val_preds)

Accuracy Score: 0.689

Classification Report               precision    recall  f1-score   support

        Neut       0.59      0.67      0.63      1313
         Pos       0.72      0.74      0.73      2005
         Neg       0.74      0.64      0.69      1682

    accuracy                           0.69      5000
   macro avg       0.69      0.68      0.68      5000
weighted avg       0.69      0.69      0.69      5000



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.639715,0.174197,0.186088
True Neut,0.127951,0.667936,0.204113
True Pos,0.101746,0.154115,0.744140


### TF - IDF Bigram

In [ ]:
tf_bi_nn_swn, history = neural_network(tf_bi_train_vecs, nn_train_swn_target, val_data = (tf_bi_val_vecs, nn_val_swn_target), 
                                model_name = 'tf_bi_nn_swn', dropout_rate = 0.1, neurons = 100, num_layers = 3, patience = 20)

Epoch 1/100
1137/1137 - 4s - loss: 0.9238 - accuracy: 0.5612 - val_loss: 0.8570 - val_accuracy: 0.6192
Epoch 2/100
1137/1137 - 3s - loss: 0.8390 - accuracy: 0.6282 - val_loss: 0.8272 - val_accuracy: 0.6388
Epoch 3/100
1137/1137 - 3s - loss: 0.7911 - accuracy: 0.6574 - val_loss: 0.8097 - val_accuracy: 0.6476
Epoch 4/100
1137/1137 - 3s - loss: 0.7415 - accuracy: 0.6844 - val_loss: 0.7984 - val_accuracy: 0.6554
Epoch 5/100
1137/1137 - 4s - loss: 0.6951 - accuracy: 0.7080 - val_loss: 0.8004 - val_accuracy: 0.6610
Epoch 6/100
1137/1137 - 3s - loss: 0.6567 - accuracy: 0.7261 - val_loss: 0.7920 - val_accuracy: 0.6670
Epoch 7/100
1137/1137 - 3s - loss: 0.6293 - accuracy: 0.7382 - val_loss: 0.7813 - val_accuracy: 0.6740
Epoch 8/100
1137/1137 - 3s - loss: 0.6062 - accuracy: 0.7475 - val_loss: 0.7986 - val_accuracy: 0.6734
Epoch 9/100
1137/1137 - 3s - loss: 0.5840 - accuracy: 0.7573 - val_loss: 0.8194 - val_accuracy: 0.6728
Epoch 10/100
1137/1137 - 3s - loss: 0.5607 - accuracy: 0.7666 - val_loss:

In [ ]:
tfbi_nn_swn_train_preds = np.argmax(tf_bi_nn_swn.predict(tf_bi_train_vecs), axis = -1) - 1
classif_reports(train_swn_target, tfbi_nn_swn_train_preds)

Accuracy Score: 0.8356111645813282

Classification Report               precision    recall  f1-score   support

        Neut       0.75      0.82      0.78      9632
         Pos       0.85      0.86      0.86     14691
         Neg       0.89      0.82      0.85     12042

    accuracy                           0.84     36365
   macro avg       0.83      0.83      0.83     36365
weighted avg       0.84      0.84      0.84     36365



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.819133,0.099319,0.081548
True Neut,0.060943,0.817068,0.121989
True Pos,0.042815,0.095909,0.861276


In [ ]:
tfbi_nn_swn_val_preds = np.argmax(tf_bi_nn_swn.predict(tf_bi_val_vecs), axis = -1) - 1
classif_reports(val_swn_target, tfbi_nn_swn_val_preds)

Accuracy Score: 0.674

Classification Report               precision    recall  f1-score   support

        Neut       0.58      0.62      0.60      1313
         Pos       0.70      0.74      0.72      2005
         Neg       0.73      0.64      0.68      1682

    accuracy                           0.67      5000
   macro avg       0.67      0.67      0.67      5000
weighted avg       0.68      0.67      0.67      5000



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.636742,0.168847,0.194411
True Neut,0.142422,0.623001,0.234577
True Pos,0.107731,0.153616,0.738653


### TF - IDF Trigram

In [ ]:
tf_tri_train_vecs = pd.read_csv('./data/tf_tri_train_vecs.csv', header = None)
tf_tri_val_vecs = pd.read_csv('./data/tf_tri_val_vecs.csv', header = None)

In [ ]:
tf_tri_nn_swn, history = neural_network(tf_tri_train_vecs, nn_train_swn_target, val_data = (tf_tri_val_vecs, nn_val_swn_target), 
                                model_name = 'tf_tri_nn_swn', dropout_rate = 0.1, neurons = 100, num_layers = 3, patience = 20)

Epoch 1/100
1137/1137 - 4s - loss: 0.9197 - accuracy: 0.5673 - val_loss: 0.8526 - val_accuracy: 0.6266
Epoch 2/100
1137/1137 - 4s - loss: 0.8352 - accuracy: 0.6329 - val_loss: 0.8299 - val_accuracy: 0.6372
Epoch 3/100
1137/1137 - 3s - loss: 0.7884 - accuracy: 0.6601 - val_loss: 0.8125 - val_accuracy: 0.6470
Epoch 4/100
1137/1137 - 3s - loss: 0.7393 - accuracy: 0.6852 - val_loss: 0.7950 - val_accuracy: 0.6608
Epoch 5/100
1137/1137 - 4s - loss: 0.6989 - accuracy: 0.7065 - val_loss: 0.7963 - val_accuracy: 0.6638
Epoch 6/100
1137/1137 - 3s - loss: 0.6603 - accuracy: 0.7250 - val_loss: 0.7941 - val_accuracy: 0.6688
Epoch 7/100
1137/1137 - 3s - loss: 0.6249 - accuracy: 0.7420 - val_loss: 0.7837 - val_accuracy: 0.6706
Epoch 8/100
1137/1137 - 3s - loss: 0.6022 - accuracy: 0.7502 - val_loss: 0.7956 - val_accuracy: 0.6718
Epoch 9/100
1137/1137 - 3s - loss: 0.5791 - accuracy: 0.7588 - val_loss: 0.7960 - val_accuracy: 0.6772
Epoch 10/100
1137/1137 - 3s - loss: 0.5643 - accuracy: 0.7661 - val_loss:

In [ ]:
tftri_nn_swn_train_preds = np.argmax(tf_tri_nn_swn.predict(tf_tri_train_vecs), axis = -1) - 1
classif_reports(train_swn_target, tftri_nn_swn_train_preds)

Accuracy Score: 0.8300288739172281

Classification Report               precision    recall  f1-score   support

        Neut       0.75      0.81      0.78      9632
         Pos       0.84      0.87      0.85     14691
         Neg       0.89      0.80      0.85     12042

    accuracy                           0.83     36365
   macro avg       0.83      0.83      0.83     36365
weighted avg       0.83      0.83      0.83     36365



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.801860,0.101810,0.096330
True Neut,0.061047,0.805025,0.133929
True Pos,0.038323,0.092165,0.869512


In [ ]:
tftri_nn_swn_val_preds = np.argmax(tf_tri_nn_swn.predict(tf_tri_val_vecs), axis = -1) - 1
classif_reports(val_swn_target, tftri_nn_swn_val_preds)

Accuracy Score: 0.6706

Classification Report               precision    recall  f1-score   support

        Neut       0.58      0.61      0.59      1313
         Pos       0.69      0.75      0.72      2005
         Neg       0.73      0.63      0.68      1682

    accuracy                           0.67      5000
   macro avg       0.67      0.66      0.66      5000
weighted avg       0.67      0.67      0.67      5000



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.631986,0.168847,0.199168
True Neut,0.135567,0.606245,0.258187
True Pos,0.107232,0.147631,0.745137


## Afin

### TF - IDF Unigram

In [ ]:
tf_uni_nn_afn, history = neural_network(tf_uni_train_vecs, nn_train_afn_target, val_data = (tf_uni_val_vecs, nn_val_afn_target),
                                        num_layers = 3, neurons = 100, dropout_rate = 0.1, model_name = 'tf_uni_nn_afn', patience = 20)

Epoch 1/100
1137/1137 - 7s - loss: 0.8189 - accuracy: 0.6311 - val_loss: 0.7099 - val_accuracy: 0.7132
Epoch 2/100
1137/1137 - 4s - loss: 0.7172 - accuracy: 0.6972 - val_loss: 0.6841 - val_accuracy: 0.7348
Epoch 3/100
1137/1137 - 4s - loss: 0.6736 - accuracy: 0.7213 - val_loss: 0.6610 - val_accuracy: 0.7364
Epoch 4/100
1137/1137 - 4s - loss: 0.6368 - accuracy: 0.7373 - val_loss: 0.6510 - val_accuracy: 0.7430
Epoch 5/100
1137/1137 - 3s - loss: 0.5939 - accuracy: 0.7574 - val_loss: 0.6336 - val_accuracy: 0.7488
Epoch 6/100
1137/1137 - 3s - loss: 0.5594 - accuracy: 0.7749 - val_loss: 0.6417 - val_accuracy: 0.7496
Epoch 7/100
1137/1137 - 3s - loss: 0.5370 - accuracy: 0.7813 - val_loss: 0.6383 - val_accuracy: 0.7522
Epoch 8/100
1137/1137 - 3s - loss: 0.5126 - accuracy: 0.7928 - val_loss: 0.6382 - val_accuracy: 0.7554
Epoch 9/100
1137/1137 - 3s - loss: 0.4908 - accuracy: 0.8026 - val_loss: 0.6477 - val_accuracy: 0.7490
Epoch 10/100
1137/1137 - 4s - loss: 0.4748 - accuracy: 0.8093 - val_loss:

In [ ]:
tfuni_train_preds_afn = np.argmax(tf_uni_nn_afn.predict(tf_uni_train_vecs), axis = -1) - 1
classif_reports(train_afn_target, tfuni_train_preds_afn)

Accuracy Score: 0.8419909253402997

Classification Report               precision    recall  f1-score   support

        Neut       0.76      0.89      0.82     12086
         Pos       0.90      0.83      0.86     11581
         Neg       0.89      0.81      0.85     12698

    accuracy                           0.84     36365
   macro avg       0.85      0.84      0.84     36365
weighted avg       0.85      0.84      0.84     36365



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.809340,0.151047,0.039613
True Neut,0.065282,0.886232,0.048486
True Pos,0.037648,0.130731,0.831621


In [ ]:
tfuni_val_preds_afn = np.argmax(tf_uni_nn_afn.predict(tf_uni_val_vecs), axis = -1) - 1
classif_reports(val_afn_target, tfuni_val_preds_afn)

Accuracy Score: 0.7488

Classification Report               precision    recall  f1-score   support

        Neut       0.66      0.79      0.72      1712
         Pos       0.81      0.74      0.77      1529
         Neg       0.81      0.71      0.76      1759

    accuracy                           0.75      5000
   macro avg       0.76      0.75      0.75      5000
weighted avg       0.76      0.75      0.75      5000



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.712905,0.225128,0.061967
True Neut,0.115070,0.794393,0.090537
True Pos,0.068018,0.192937,0.739045



### TF - IDF Bigram


In [ ]:
tf_bi_nn_afn, history = neural_network(tf_bi_train_vecs, nn_train_afn_target, val_data = (tf_bi_val_vecs, nn_val_afn_target),
                                        num_layers = 3, neurons = 100, dropout_rate = 0.1, model_name = 'tf_bi_nn_afn', patience = 20)

Epoch 1/100
1137/1137 - 4s - loss: 0.8265 - accuracy: 0.6258 - val_loss: 0.7255 - val_accuracy: 0.7026
Epoch 2/100
1137/1137 - 3s - loss: 0.7322 - accuracy: 0.6882 - val_loss: 0.7033 - val_accuracy: 0.7144
Epoch 3/100
1137/1137 - 3s - loss: 0.6929 - accuracy: 0.7081 - val_loss: 0.6753 - val_accuracy: 0.7316
Epoch 4/100
1137/1137 - 3s - loss: 0.6484 - accuracy: 0.7313 - val_loss: 0.6646 - val_accuracy: 0.7292
Epoch 5/100
1137/1137 - 3s - loss: 0.6131 - accuracy: 0.7470 - val_loss: 0.6769 - val_accuracy: 0.7232
Epoch 6/100
1137/1137 - 4s - loss: 0.5771 - accuracy: 0.7633 - val_loss: 0.6529 - val_accuracy: 0.7432
Epoch 7/100
1137/1137 - 3s - loss: 0.5514 - accuracy: 0.7742 - val_loss: 0.6521 - val_accuracy: 0.7402
Epoch 8/100
1137/1137 - 3s - loss: 0.5268 - accuracy: 0.7823 - val_loss: 0.6494 - val_accuracy: 0.7518
Epoch 9/100
1137/1137 - 3s - loss: 0.5071 - accuracy: 0.7915 - val_loss: 0.6458 - val_accuracy: 0.7516
Epoch 10/100
1137/1137 - 3s - loss: 0.4911 - accuracy: 0.7988 - val_loss:

In [ ]:
tfbi_train_preds_afn = np.argmax(tf_bi_nn_afn.predict(tf_bi_train_vecs), axis = -1) - 1
classif_reports(train_afn_target, tfbi_train_preds_afn)

Accuracy Score: 0.8770521105458545

Classification Report               precision    recall  f1-score   support

        Neut       0.78      0.94      0.86     12086
         Pos       0.94      0.86      0.90     11581
         Neg       0.93      0.83      0.88     12698

    accuracy                           0.88     36365
   macro avg       0.89      0.88      0.88     36365
weighted avg       0.89      0.88      0.88     36365



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.834383,0.140810,0.024807
True Neut,0.038061,0.939434,0.022505
True Pos,0.026595,0.114671,0.858734


In [ ]:
tfbi_val_preds_afn = np.argmax(tf_bi_nn_afn.predict(tf_bi_val_vecs), axis = -1) - 1
classif_reports(val_afn_target, tfbi_val_preds_afn)

Accuracy Score: 0.7516

Classification Report               precision    recall  f1-score   support

        Neut       0.66      0.81      0.73      1712
         Pos       0.82      0.73      0.78      1529
         Neg       0.82      0.71      0.76      1759

    accuracy                           0.75      5000
   macro avg       0.77      0.75      0.75      5000
weighted avg       0.77      0.75      0.75      5000



,Pred Neg,Pred Neut,Pred Pos
True Neg,0.706652,0.236498,0.056850
True Neut,0.105724,0.813668,0.080607
True Pos,0.060170,0.206017,0.733813



### TF - IDF Trigram


In [ ]:
tf_tri_nn_afn, history = neural_network(tf_tri_train_vecs, nn_train_afn_target, val_data = (tf_tri_val_vecs, nn_val_afn_target),
                                        num_layers = 3, neurons = 100, dropout_rate = 0.1, model_name = 'tf_tri_nn_afn', patience = 20)

Epoch 1/100
1137/1137 - 4s - loss: 0.8315 - accuracy: 0.6204 - val_loss: 0.7329 - val_accuracy: 0.6984
Epoch 2/100
1137/1137 - 4s - loss: 0.7336 - accuracy: 0.6886 - val_loss: 0.7058 - val_accuracy: 0.7026
Epoch 3/100
1137/1137 - 3s - loss: 0.6882 - accuracy: 0.7113 - val_loss: 0.6741 - val_accuracy: 0.7318
Epoch 4/100
1137/1137 - 3s - loss: 0.6433 - accuracy: 0.7338 - val_loss: 0.6595 - val_accuracy: 0.7314
Epoch 5/100
1137/1137 - 3s - loss: 0.6044 - accuracy: 0.7510 - val_loss: 0.6570 - val_accuracy: 0.7294
Epoch 6/100
1137/1137 - 3s - loss: 0.5713 - accuracy: 0.7654 - val_loss: 0.6458 - val_accuracy: 0.7384
Epoch 7/100
1137/1137 - 4s - loss: 0.5434 - accuracy: 0.7775 - val_loss: 0.6409 - val_accuracy: 0.7432
Epoch 8/100
1137/1137 - 4s - loss: 0.5207 - accuracy: 0.7880 - val_loss: 0.6543 - val_accuracy: 0.7338
Epoch 9/100
1137/1137 - 3s - loss: 0.4980 - accuracy: 0.7974 - val_loss: 0.6699 - val_accuracy: 0.7322
Epoch 10/100
1137/1137 - 3s - loss: 0.4859 - accuracy: 0.8051 - val_loss:

In [ ]:
tftri_train_preds_afn = np.argmax(tf_tri_nn_afn.predict(tf_tri_train_vecs), axis = -1) - 1
classif_reports(train_afn_target, tftri_train_preds_afn)

NameError: name 'np' is not defined

In [ ]:
tftri_val_preds_afn = np.argmax(tf_tri_nn_afn.predict(tf_tri_val_vecs), axis = -1) - 1
classif_reports(val_afn_target, tftri_val_preds_afn)

NameError: name 'np' is not defined

## Recurrent Neural Network

In [15]:
train_vecs = tf_uni_train_vecs
val_vecs = tf_uni_val_vecs

In [16]:
train_vecs_tensor = tf.convert_to_tensor(train_vecs, dtype = tf.float32)
train_vecs_tensor = tf.expand_dims(train_vecs_tensor, axis = -1)

val_vecs_tensor = tf.convert_to_tensor(val_vecs, dtype = tf.float32)
val_vecs_tensor = tf.expand_dims(val_vecs_tensor, axis = -1)

print(train_vecs_tensor.shape, val_vecs_tensor.shape)

(36365, 500, 1) (5000, 500, 1)


In [17]:
def recurrent_nn(X, y, val_data = None, num_layers = 3, neurons = 100, recurrent_neurons = 50, 
                    dropout_rate = 0.1, rec_drop_rate = 0.1, model_name = 'rnn'):

    model = models.Sequential()

    model.add(layers.Dense(neurons, activation = 'relu', input_shape = train_vecs_tensor.shape[1:]))

    for layer in range(num_layers):
        ret_seq = True
        if layer == num_layers - 1:
            ret_seq = False
        model.add(layers.GRU(recurrent_neurons, return_sequences = ret_seq, recurrent_dropout = rec_drop_rate, dropout = dropout_rate))

    model.add(layers.Dense(3, activation = 'softmax'))

    model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    
    run_log_dir = get_run_logdir(model_name + 'num_layers_' + str(num_layers) + '_rec_neurons_'
                                     + str(recurrent_neurons) + '_drop_rate_' + str(dropout_rate))

    checkpoint_cb = callbacks.ModelCheckpoint('./models/nn models' + model_name + '.h5', save_best_only = True)
    early_stop_cb = callbacks.EarlyStopping(patience = 5, restore_best_weights = True)
    tensorboard_cb = callbacks.TensorBoard(run_log_dir)

    model.fit(X, y, validation_data = val_data,
                        epochs = 100, verbose = 2, batch_size = 64,
                        callbacks = [checkpoint_cb, tensorboard_cb, early_stop_cb])


    return model

In [18]:
rnn_swn = recurrent_nn(X = train_vecs_tensor, y = nn_train_swn_target, val_data = (val_vecs_tensor, nn_val_swn_target), 
                        num_layers = 1, neurons = 100, recurrent_neurons = 50, dropout_rate = 0.1, rec_drop_rate = 0.1)

Epoch 1/100
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
569/569 - 633s - loss: 1.0849 - accuracy: 0.4038 - val_loss: 1.0836 - val_accuracy: 0.4010
Epoch 2/100
569/569 - 629s - loss: 1.0837 - accuracy: 0.4041 - val_loss: 1.0831 - val_accuracy: 0.4014
Epoch 3/100
569/569 - 635s - loss: 1.0824 - accuracy: 0.4040 - val_loss: 1.0826 - val_accuracy: 0.4026
Epoch 4/100
569/569 - 609s - loss: 1.0811 - accuracy: 0.4039 - val_loss: 1.0816 - val_accuracy: 0.4028
Epoch 5/100
569/569 - 612s - loss: 1.0784 - accuracy: 0.4043 - val_loss: 1.0789 - val_accuracy: 0.4032
Epoch 6/100
569/569 - 634s - loss: 1.0734 - accuracy: 0.4065 - val_loss: 1.0744 - val_accuracy: 0.4116
Epoch 7/100
569/569 - 625s - loss: 1.0776 - accuracy: 0.4042 - val_loss: 1.0679 - val_accuracy: 0.4158
Epoch 8/100
569/569 - 629s - loss: 1.0626 - accuracy: 0.4192 - val_loss: 1.0628 - val_accuracy: 0.4160
Epoch 9/100
569/569 - 635s - loss: 1.0577 - accuracy: 0.4245 - val_loss: 1.0607 - val_accuracy: 0.4142
E

KeyboardInterrupt: 

In [ ]:
rnn_train_preds_probs = rnn_swn.predict(train_vecs_tensor)
rnn_train_preds_classes = np.argmax(rnn_train_preds_probs, axis = -1)

In [ ]:
rnn_train_preds_classes

In [ ]:
classif_reports(train_swn_target, rnn_train_preds_classes)

Accuracy Score: 0.13702736147394473

Classification Report               precision    recall  f1-score   support

        Neut       0.17      0.22      0.19      9632
         Pos       0.29      0.19      0.23     14691
         Neg       0.00      0.00      0.00     12042

   micro avg       0.22      0.14      0.17     36365
   macro avg       0.15      0.14      0.14     36365
weighted avg       0.16      0.14      0.15     36365

/opt/venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ValueError: Shape of passed values is (4, 4), indices imply (3, 3)

In [ ]:
rnn_val_preds_probs = rnn_swn.predict(val_vecs_tensor)
rnn_val_preds_classes = np.argmax(rnn_val_preds_probs, axis = -1)

In [ ]:
classif_reports(val_swn_target, rnn_val_preds_classes)